Andre Telfer

# REST APIs for Scrubs
This is a notebook for showing how to get stock data using python.

working with REST APIs is ridiculously easy in python. REST APIs normally work by taking in a variables in the URL Path/Parameters - you then use the url+params to send a get request which returns the data you want (often in JSON format).

Often, they require a 'key' to be passed in the URL Parameters, however IEX does not right now

# Resources
Basically you just read the docs for whatever REST API you're using to figure out what the URL path should look like, and what params you need

IEX is a REST API for getting stock data. [docs](https://iextrading.com/developer/docs/)


In [ ]:
import requests
import json

def get_stock_chart(company):
    '''
    Gets the last 6m of data for a given company
    
    :param company: the company to request data for 
    '''
    
    # API URL Path, right now this gets 6 months of 'chart' data for a given company. 
    # The API docs describe chart data as a time series
    url = "https://api.iextrading.com/1.0/stock/{company}/chart/6m".format(
        company = company
    )
    
    # API URL Params
    params = None # we don't need any params, but this is how you'd do it if you did
    
    # send a REST get request to the URL Path with the given URL Params, save the response
    resp = requests.get(url, params)
    
    # make sure the request was successful 
    assert resp.status_code == requests.status_codes.codes.OK
    
    # return the text body of the response
    return json.loads(resp.text)

# try getting 6 of data for aapl
data = get_stock_chart('aapl')

# print some of it prettily (indent lists/dictionaries)
print(json.dumps(data, indent=4))

# Working with the Data

I mean, you could do this however you want.  

Pandas is pretty nice, so that's what I use. There can be a bit of a learning curve to take full advantage of it. But I'm not an expert on it and it's already very useful.
- read the docs, use stackoverflow. Pandas is very popular

In [36]:
import pandas as pd
from IPython.display import display

# a dataframe is a convenient way to work with data
df = pd.DataFrame(data)

# lets try displaying the last 3 data points
display(df.tail(3))

# lets try selecting a column and displaying its last 5 data points
# the 'high' column is from the REST API and is the highest value of the stock in the day
df = df.set_index('date') # just going to set the index as the date 
display(df.high.tail(5))

# lets take the rolling average of the high column to smooth it out a bit 
# The rolling mean is the average of a value and some number of previous values. 
r = df.high.rolling(window=6) # window is the number of previous values
display(r.mean().tail(5))

,change,changeOverTime,changePercent,close,date,high,label,low,open,unadjustedVolume,volume,vwap
122,3.03,0.064627,1.844,167.37,2018-02-14,167.54,Feb 14,162.88,163.045,40644933,40644933,165.5906
123,5.62,0.100375,3.358,172.99,2018-02-15,173.09,Feb 15,169.00,169.790,51147171,51147171,171.4911
124,-0.56,0.096813,-0.324,172.43,2018-02-16,174.82,Feb 16,171.77,172.360,40176091,40176091,173.4476


date
2018-02-12    163.89
2018-02-13    164.75
2018-02-14    167.54
2018-02-15    173.09
2018-02-16    174.82
Name: high, dtype: float64

date
2018-02-12    162.296667
2018-02-13    162.441667
2018-02-14    163.078333
2018-02-15    164.693333
2018-02-16    166.996667
Name: high, dtype: float64